<a href="https://colab.research.google.com/github/mmfara/Disparate-Impact-Remover-Enhanced/blob/main/DIR%2B_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
!pip install aif360
!pip install aif360[LawSchoolGPA]
!pip install aif360[Reductions]
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import DisparateImpactRemover
!pip install BlackBoxAuditing

pip install 'aif360[inFairness]'


In [2]:
# Import necessary libraries
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions import get_distortion_adult
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from aif360.algorithms.inprocessing import ExponentiatedGradientReduction
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.metrics import ClassificationMetric

#STARTING OFF

In [3]:
data = pd.read_csv("/content/adult_sex_race_columns.csv")

In [4]:
data.head()

,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,income,age_group,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia,sex,race
0,77053,9,0,4356,40,0,3,0,0,1,...,0,0,0,0,0,1,0,0,0,1
1,132870,9,0,4356,18,0,3,0,0,1,...,0,0,0,0,0,1,0,0,0,1
2,186061,10,0,4356,40,0,3,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,140359,4,0,3900,40,0,3,0,0,1,...,0,0,0,0,0,1,0,0,0,1
4,264663,10,0,3900,40,0,2,0,0,1,...,0,0,0,0,0,1,0,0,0,1


In [5]:
X = data.drop(['income'], axis =1)
y = data[['income']]

In [6]:
# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle =True, test_size=0.4, random_state=42)

In [7]:
print(X_train.shape)
print(X_test.shape)

(18550, 78)
(12367, 78)


In [8]:
# Create training DataFrame
train_df = X_train.copy()
train_df['income'] = y_train

test_df = X_test.copy()
test_df['income'] = y_test

In [9]:
# Group by race and gender, then label
group_label_counts = train_df.groupby(['race', 'sex', 'income']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts)


          Negative (0)  Positive (1)
race sex                            
0    0             837            50
     1             739           178
1    0            4623           611
     1            7850          3662


In [10]:
# Check value counts by group and label
group_label_counts_test = test_df.groupby(['race', 'sex', 'income']).size().unstack(fill_value=0)

# Optional: Rename columns for clarity
group_label_counts_test.columns = ['Negative (0)', 'Positive (1)']

# Display
print(group_label_counts_test)


          Negative (0)  Positive (1)
race sex                            
0    0             628            40
     1             531           119
1    0            2982           417
     1            5225          2425


In [11]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler, SMOTE

# Define target and protected attribute columns
target_col = 'income'
protected_cols = ['race', 'sex']  # Intersectional attributes

# Add label back to training data temporarily
X_train_temp = X_train.copy()
X_train_temp[target_col] = y_train

# Create a composite group identifier
X_train_temp['group'] = X_train_temp[protected_cols].astype(str).agg('_'.join, axis=1)

# Pick a sampler
sampler = RandomOverSampler(random_state=42)
# sampler = SMOTE(random_state=42)

resampled_X = []
resampled_y = []

# Oversample within each intersectional group
for group in X_train_temp['group'].unique():
    group_data = X_train_temp[X_train_temp['group'] == group]

    Xg = group_data.drop(columns=[target_col])
    yg = group_data[target_col]

    # Drop group before resampling
    Xg_no_group = Xg.drop(columns=['group'], errors='ignore')

    if yg.nunique() < 2:
        print(f"Skipping group {group}: only one class present.")
        continue

    Xg_res, yg_res = sampler.fit_resample(Xg_no_group, yg)

    # Restore the group column after resampling
    Xg_res['group'] = group
    resampled_X.append(Xg_res)
    resampled_y.append(pd.Series(yg_res))

# Combine everything
X_train_bal = pd.concat(resampled_X).reset_index(drop=True)
y_train_bal = pd.concat(resampled_y).reset_index(drop=True)

# Reattach target label
X_train_bal[target_col] = y_train_bal

# ✅ View final balanced counts per intersectional group
print("✅ Final balanced class counts per (race, sex):")
X_train_bal[['group', target_col]].groupby(['group', target_col]).size().unstack(fill_value=0)


✅ Final balanced class counts per (race, sex):


income,0,1
group,,
0_0,837,837
0_1,739,739
1_0,4623,4623
1_1,7850,7850


In [12]:
X_train_bal.drop(columns=['group'], inplace=True)

In [13]:
dataset_train = X_train_bal.reset_index(drop=True)
dataset_test = test_df.reset_index(drop=True)

In [14]:
dataset_train.shape

(28098, 79)

In [15]:
dataset_test.shape

(12367, 79)

Checking my datasets

In [16]:
dataset_train

,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,age_group,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,...,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia,sex,race,income
0,196396,10,0,0,40,1,0,0,1,0,...,0,0,0,0,1,0,0,1,1,0
1,103435,9,0,0,50,2,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
2,209547,13,0,0,60,2,0,0,1,0,...,0,0,0,0,1,0,0,1,1,1
3,174666,6,0,0,80,1,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
4,255847,9,0,0,40,2,1,0,0,0,...,0,0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28093,99254,12,0,0,40,2,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
28094,55743,13,0,0,45,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
28095,172855,13,0,1887,40,2,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1
28096,348491,10,0,0,40,2,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [17]:
# Define favorable and unfavorable labels
favorable_label = 1
unfavorable_label = 0

privileged_groups = [{'race': 1, 'sex': 1}]

unprivileged_groups = [
    {'race': r, 'sex': s}
    for r in data['race'].unique()
    for s in data['sex'].unique()
    if not (r == 1 and s == 1)]


#DisparateImpactRemover

In [18]:
import numpy as np
import logging
from typing import List, Union, Set
from aif360.algorithms import Transformer


class DisparateImpactRemover(Transformer):
    """
    Enhanced Disparate Impact Remover with:
    - Intersectionality support
    - min_group_size filtering
    - groups_to_repair selective control
    - Global repair (single Repairer call)
    - Robust verbose logging
    """

    def __init__(self,
                 repair_level: float = 1.0,
                 sensitive_attribute: Union[str, List[str], None] = None,
                 min_group_size: int = 0,
                 groups_to_repair: Union[List[str], Set[str], None] = None,
                 verbose: bool = True):
        super().__init__()
        from BlackBoxAuditing.repairers.GeneralRepairer import Repairer
        self.Repairer = Repairer

        if not 0.0 <= repair_level <= 1.0:
            raise ValueError("'repair_level' must be between 0.0 and 1.0.")
        self.repair_level = repair_level
        self.min_group_size = max(0, min_group_size)
        self.groups_to_repair = set(groups_to_repair) if groups_to_repair else None
        self.verbose = verbose

        # Handle sensitive attributes
        if sensitive_attribute is None:
            self.sensitive_attributes = []
        elif isinstance(sensitive_attribute, str):
            self.sensitive_attributes = [sensitive_attribute]
        elif isinstance(sensitive_attribute, list):
            self.sensitive_attributes = sensitive_attribute
        else:
            raise TypeError("sensitive_attribute must be str, list, or None")

        # Robust logging setup
        self.logger = logging.getLogger("DisparateImpactRemover")
        if self.verbose:
            self.logger.setLevel(logging.INFO)
            if not self.logger.hasHandlers():
                handler = logging.StreamHandler()
                formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
                handler.setFormatter(formatter)
                self.logger.addHandler(handler)

    def fit_transform(self, dataset):
        features = dataset.features.copy()
        repaired = dataset.copy()

        if not self.sensitive_attributes:
            self.sensitive_attributes = dataset.protected_attribute_names[:1]

        protected_indices = [dataset.feature_names.index(attr) for attr in self.sensitive_attributes]

        # Generate readable group labels like "group=1" or "gender=0|race=1"
        combined_groups = self._get_group_labels(features, dataset)
        unique, counts = np.unique(combined_groups, return_counts=True)

        if self.verbose:
            self._log_group_stats(unique, counts)

        # Determine which groups to include
        valid_groups = []
        for grp, cnt in zip(unique, counts):
            reason = []
            if cnt < self.min_group_size:
                reason.append(f"size {cnt} < {self.min_group_size}")
            if self.groups_to_repair is not None and grp not in self.groups_to_repair:
                reason.append("not in groups_to_repair")

            if reason:
                self.logger.info(f"Skipped group: {grp} — {'; '.join(reason)}")
            else:
                valid_groups.append(grp)
                self.logger.info(f"Included group: {grp} (size={cnt})")

        # Select only rows from valid groups
        group_mask = np.isin(combined_groups, valid_groups)
        if not np.any(group_mask):
            raise ValueError("No valid rows to repair after filtering.")

        filtered_features = features[group_mask]
        filtered_labels = combined_groups[group_mask]

        # Map valid group labels to numeric codes
        label_to_code = {label: i for i, label in enumerate(sorted(set(filtered_labels)))}
        group_codes = np.array([label_to_code[label] for label in filtered_labels])

        # Append group code to filtered features
        repair_input = np.hstack([filtered_features, group_codes.reshape(-1, 1)])
        repair_index = repair_input.shape[1] - 1

        # Run Repairer
        repaired_filtered = np.array(
            self.Repairer(repair_input.tolist(), repair_index, self.repair_level, False).repair(repair_input.tolist()),
            dtype=np.float64
        )

        # FIXED: Apply repair only to valid rows (no shape mismatch!)
        repaired_features = features.copy()
        repaired_features[group_mask] = repaired_filtered[:, :-1]

        # Restore protected attributes
        for idx in protected_indices:
            repaired_features[:, idx] = dataset.features[:, idx]

        repaired.features = repaired_features
        return repaired

    def _get_group_labels(self, features: np.ndarray, dataset) -> np.ndarray:
        """Create human-readable group labels from protected attributes"""
        indices = [dataset.feature_names.index(attr) for attr in self.sensitive_attributes]
        protected_names = [dataset.feature_names[idx] for idx in indices]
        return np.array([
            '|'.join(f"{protected_names[j]}={features[i, idx]}"
                     for j, idx in enumerate(indices))
            for i in range(features.shape[0])
        ])

    def _log_group_stats(self, unique, counts):
        """Print stats for all groups"""
        self.logger.info("\n=== Group Analysis ===")
        self.logger.info(f"Protected attributes: {self.sensitive_attributes}")
        self.logger.info(f"Minimum group size: {self.min_group_size}")
        if self.groups_to_repair:
            self.logger.info(f"Specific groups to repair: {self.groups_to_repair}")
        self.logger.info(f"\n{'Status'.ljust(12)} {'Group'.ljust(40)} Size")
        for grp, cnt in sorted(zip(unique, counts), key=lambda x: -x[1]):
            status = []
            if cnt < self.min_group_size:
                status.append("TOO SMALL")
            if self.groups_to_repair and grp not in self.groups_to_repair:
                status.append("NOT SELECTED")
            status_str = "|".join(status) if status else "PROCESS"
            self.logger.info(f"{status_str.ljust(12)} {grp.ljust(40)} {cnt}")


In [19]:
# Instantiate the enhanced DIR
dir_global = DisparateImpactRemover(
    repair_level=1.0,                        # strength of repair
    sensitive_attribute=["race", "sex"],  # intersectional attributes
    min_group_size=30                       # optional: skip small groups
)


groups_to_repair={"race=1|gender=0", "race=1|gender=1"}  # optional: selective repair

In [21]:
dataset_train_bld = BinaryLabelDataset(df=dataset_train,
                             label_names=['income'],
                             favorable_label=favorable_label,
                             unfavorable_label=unfavorable_label,
                             protected_attribute_names=['race', 'sex'],
                             privileged_protected_attributes=privileged_groups)


dataset_test_bld = BinaryLabelDataset(df=dataset_test,
                                      label_names=['income'],
                                      favorable_label=favorable_label,
                                      unfavorable_label=unfavorable_label,
                                      protected_attribute_names=['race', 'sex'],
                                      privileged_protected_attributes=privileged_groups)

###Fitting disparate impact removal on the dataset

In [22]:
# Fit and transform the training data
dirglobal = dir_global.fit_transform(dataset_train_bld)

INFO:DisparateImpactRemover:
=== Group Analysis ===
INFO:DisparateImpactRemover:Protected attributes: ['race', 'sex']
INFO:DisparateImpactRemover:Minimum group size: 30
INFO:DisparateImpactRemover:
Status       Group                                    Size
INFO:DisparateImpactRemover:PROCESS      race=1.0|sex=1.0                         15700
INFO:DisparateImpactRemover:PROCESS      race=1.0|sex=0.0                         9246
INFO:DisparateImpactRemover:PROCESS      race=0.0|sex=0.0                         1674
INFO:DisparateImpactRemover:PROCESS      race=0.0|sex=1.0                         1478
INFO:DisparateImpactRemover:Included group: race=0.0|sex=0.0 (size=1674)
INFO:DisparateImpactRemover:Included group: race=0.0|sex=1.0 (size=1478)
INFO:DisparateImpactRemover:Included group: race=1.0|sex=0.0 (size=9246)
INFO:DisparateImpactRemover:Included group: race=1.0|sex=1.0 (size=15700)


In [23]:
# Define features and target variable
X_train_global = dirglobal.features
y_train_global = dirglobal.labels.ravel()

X_test = dataset_test_bld.features
y_test = dataset_test_bld.labels.ravel()

In [24]:
scaler_global = StandardScaler()
X_train_scaledglobal = scaler_global.fit_transform(X_train_global)
X_test_scaled = scaler_global.transform(X_test)

X_train_scaledglobal = pd.DataFrame(X_train_scaledglobal, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

y_trainglobal = pd.DataFrame(y_train_global, columns=y.columns)
y_test_df = pd.DataFrame(y_test, columns=y.columns)

dataset_train_global = pd.concat([X_train_scaledglobal, y_trainglobal], axis=1)
dataset_test = pd.concat([X_test_scaled, y_test_df], axis=1)

#Model

In [27]:
model_global = LogisticRegression()

In [28]:
model_global.fit(X_train_scaledglobal, y_trainglobal['income'])

LogisticRegression()

In [31]:
y_pred_global = model_global.predict(X_test_scaled)

In [32]:
accuracy_score(y_pred_global, y_test)

0.7918654483706639

In [35]:
y_pred_global = pd.DataFrame(y_pred_global, columns=['income'])

In [36]:
X_test = pd.DataFrame(X_test, columns=X.columns)

In [37]:
y_test_df = pd.DataFrame(y_test, columns=['income'])
dataset_test = pd.concat([X_test, y_test_df], axis=1)


dataset_testpred_global = pd.concat([X_test, y_pred_global], axis=1)

In [39]:
# Convert DataFrames back to AIF360 BinaryLabelDataset
dataset_test = BinaryLabelDataset(df=dataset_test,
                                          label_names=['income'],
                                          favorable_label=favorable_label,
                                          unfavorable_label=unfavorable_label,
                                          protected_attribute_names=['race', 'sex'],
                                          privileged_protected_attributes=privileged_groups)

dataset_testpred_global = BinaryLabelDataset(df=dataset_testpred_global,
                                         label_names=['income'],
                                         favorable_label=favorable_label,
                                         unfavorable_label=unfavorable_label,
                                         protected_attribute_names=['race', 'sex'],
                                         privileged_protected_attributes=privileged_groups)

#Global Results

In [40]:
# Evaluate the classifier's performance with fairness intervention
metric_with_fairness = ClassificationMetric(
    dataset_test,
    dataset_testpred_global,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print("Performance with fairness intervention (Reject Option Classification):")
print("Accuracy: {:.6f}".format(metric_with_fairness.accuracy()))
print("Disparate Impact: {:.6f}".format(metric_with_fairness.disparate_impact()))
print("Mean Difference: {:.6f}".format(metric_with_fairness.mean_difference()))

Performance with fairness intervention (Reject Option Classification):
Accuracy: 0.791865
Disparate Impact: 0.471564
Mean Difference: -0.255860


In [41]:
# Calculate fairness metrics
statistical_parity_difference = metric_with_fairness.statistical_parity_difference()
disparate_impact = metric_with_fairness.disparate_impact()
equal_opportunity_difference = metric_with_fairness.equal_opportunity_difference()

In [42]:
# Print out the metrics
print(f"Statistical Parity Difference: {statistical_parity_difference}")
print(f"Disparate Impact: {disparate_impact}")
print(f"Equal Opportunity Difference: {equal_opportunity_difference}")

Statistical Parity Difference: -0.25585991982829454
Disparate Impact: 0.4715636105058172
Equal Opportunity Difference: -0.027503579610538287


In [43]:
# Print out the equality of odds metrics
print("Difference in True Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.true_positive_rate_difference())
print("Difference in False Positive Rates (Unprivileged - Privileged) = %f" % metric_with_fairness.false_positive_rate_difference())

Difference in True Positive Rates (Unprivileged - Privileged) = -0.027504
Difference in False Positive Rates (Unprivileged - Privileged) = -0.161254


#Intersectional result

In [44]:
import pandas as pd
import numpy as np
from itertools import product
from aif360.metrics import ClassificationMetric

# Assumes you defined this earlier:
# privileged_groups = [{'race': 1, 'sex': 1}]

# Get indices of protected attributes
race_idx = dataset_test.feature_names.index('race')
sex_idx = dataset_test.feature_names.index('sex')

# All intersectional (race, sex) combinations
race_vals = np.unique(dataset_test.features[:, race_idx])
sex_vals = np.unique(dataset_test.features[:, sex_idx])
group_combinations = list(product(race_vals, sex_vals))

# Collect results
results = []

for (race_val, sex_val) in group_combinations:
    current_group = {'race': race_val, 'sex': sex_val}

    if current_group in privileged_groups:
        continue  # Skip privileged group

    metric = ClassificationMetric(
        dataset_test,
        dataset_testpred_global,
        unprivileged_groups=[current_group],
        privileged_groups=privileged_groups
    )

    # Manual subgroup accuracy
    mask = (
        (dataset_test.features[:, race_idx] == race_val) &
        (dataset_test.features[:, sex_idx] == sex_val)
    )
    y_true = dataset_test.labels[mask]
    y_pred = dataset_testpred_global.labels[mask]
    accuracy = (y_true == y_pred).mean()

    results.append({
        'race': int(race_val),
        'sex': int(sex_val),
        'Statistical Parity Difference': metric.statistical_parity_difference(),
        'Equal Opportunity Difference': metric.equal_opportunity_difference(),
        'Disparate Impact': metric.disparate_impact(),
        'Predictive Equality Difference': metric.false_positive_rate_difference(),
        'Subgroup Accuracy': round(accuracy, 6)
    })

# Results table
df_results = pd.DataFrame(results)
display(df_results)


,race,sex,Statistical Parity Difference,Equal Opportunity Difference,Disparate Impact,Predictive Equality Difference,Subgroup Accuracy
0,0,0,-0.326997,-0.097990,0.324641,-0.185898,0.875749
1,0,1,-0.182645,-0.015847,0.622778,-0.126708,0.829231
2,1,0,-0.255881,-0.024069,0.471521,-0.162216,0.857311
